# Sphere function, vectorized

In the previous example, we solved the constrained Rosenbrock problem. This was a 2-dimensional problem, so we created two variables: $x$ and $y$.

However, imagine we had a problem with 100 variables. It'd be pretty tedious to create these variables individually and do the math on each variable one-by-one. (Not to mention the fact that it'd be slow - rule #1 of scientific Python is to vectorize everything!)

So, what we can do instead is (you guessed it) create variables that are vectors. Think of a vectorized variable as a box that contains $n$ entries, each of which is a scalar variable.

Let's demonstrate this by finding the minimum of the n-dimensional sphere problem. The sphere problem, mathematically, is a simple quadratic program:

$$ \underset{x}{\text{minimize }} \sum x_i^2 $$

In [1]:
import aerosandbox as asb
import aerosandbox.numpy as np # Whoa! What is this? Why are we writing this instead of `import numpy as np`? Don't worry, we'll talk about this in the next tutorial :)

N = 100 # Let's optimize in 100-dimensional space.

opti = asb.Opti()

# Define optimization variables
x = opti.variable(
    init_guess=np.ones(shape=N) # Creates a variable with an initial guess that is [1, 1, 1, 1,...] with N entries.
) # Note that the fact that we're declaring a vectorized variable was *inferred* automatically the shape of our initial guess.

# Define objective
f = np.sum(x ** 2)
opti.minimize(f)

# Optimize
sol = opti.solve()

# Extract values at the optimum
x_opt = sol.value(x)

# Print values
print(f"x = {x_opt}")

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      100

Total number of variables............................:      100
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1

We find the solution of this optimization problem to be a vector of 100 zeroes - makes sense.

Note that because this is an unconstrained quadratic program and we're using a modern second-order optimizer (IPOPT) as the backend, this solves in just one iteration (Newton step with $\alpha=1$).

Let's talk a bit more about vectorized variables. We demonstrated that one can create a vectorized variable with the syntax:

```python
x = opti.variable(
    init_guess=np.ones(shape=N)
)
```

One can also use the syntax:

```python
x = opti.variable(
    init_guess=1,
    n_vars=N,
)
```

Which will also initialize a vector variable of length $N$ with initial guess of 1. Of course, let's say that you wanted to initialize each element of the vector $x$ to a different value; say, something like `np.linspace(0, 1, N)`. Then, you would have to use the syntax from the first example rather than that from the second.

## A Note on Initial Guesses

Note that when solving high-dimensional, nonlinear, nonconvex systems, it is very, very important to provide an initial guess that is as close to accurate as possible! This is true for both scalar and vector variables.

## A Note on N-Dimensional Arrays of Optimization Variables

Here, we demonstrated how 1-dimensional arrays of variables can be used in optimization with AeroSandbox.

For simple analysis (without tracing derivatives), `aerosandbox.numpy` has all of the n-dimensional array capabilities of NumPy. However, in optimization (where we are tracing derivatives), only scalars, 1D, and 2D arrays are supported for now - but really, scalars and 1D arrays are usually the only things you ever need to use as optimization variables for engineering design optimization. (2D arrays of decision variables can occasionally appear in operations research problems such as the [transportation problem](https://personal.utdallas.edu/~scniu/OPRE-6201/documents/TP1-Formulation.pdf), but this usually doesn't occur in engineering design problems.)